In [1]:
import sys, random
sys.path.insert(0, '../../')
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler

import torch
from torch import optim
from kernels.nn import ImplicitDenseNetKernel
from model.ick import ICK
from model.cmick import CMICK
from benchmarks.cmgp_modified import CMGP
from benchmarks.cevae_modified import *
from benchmarks.x_learner import X_Learner_RF, X_Learner_BART
from ganite import Ganite
from utils.train import CMICKEnsembleTrainer
from utils.helpers import *
from utils.metrics import policy_risk_binary

# To make this notebook's output stable across runs
random.seed(2020)
np.random.seed(2020)
torch.manual_seed(2020)
torch.cuda.manual_seed(2020)
torch.cuda.manual_seed_all(2020)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# 1. Load and preprocess data

In [2]:
def load_and_preprocess_data(train_ratio, test_ratio, random_state):
    randomized_data_dir = '../../data/JOBS/randomized.csv'
    randomized_df = pd.read_csv(randomized_data_dir)
    nonrandomized_data_dir = '../../data/JOBS/nonrandomized.csv'
    nonrandomized_df = pd.read_csv(nonrandomized_data_dir)
    df = pd.concat([randomized_df, nonrandomized_df], ignore_index=True)
    cols_to_normalize = ['Age', 'Education', 'RE75']
    for c in df.columns:
        if c in cols_to_normalize:
            scaler = StandardScaler()
            df[c] = scaler.fit_transform(df[c].to_numpy().reshape(-1,1)).reshape(-1)
    df['RE78'] = df['RE78'].apply(lambda x: 1. if x > 0. else 0.)   # Trasform to binary classification task
    
    N = len(df)
    # Test set only from randomized samples
    test_df = df[:len(randomized_df)].sample(n=int(test_ratio*N), random_state=random_state)  
    df = df.drop(test_df.index)
    train_df = df.sample(n=int(train_ratio*N), random_state=random_state)
    val_df = df.drop(train_df.index)
    X_train, X_val, X_test = train_df.to_numpy()[:,1:-1], val_df.to_numpy()[:,1:-1], test_df.to_numpy()[:,1:-1]
    T_train, T_val, T_test = train_df.to_numpy()[:,:1], val_df.to_numpy()[:,:1], test_df.to_numpy()[:,:1]
    Y_train, Y_val, Y_test = train_df.to_numpy()[:,-1:], val_df.to_numpy()[:,-1:], test_df.to_numpy()[:,-1:]
    
    data = {'X_train': X_train, 'T_train': T_train, 'Y_train': Y_train, 'X_val': X_val, 'T_val': T_val,
            'Y_val': Y_val, 'X_test': X_test, 'T_test': T_test, 'Y_test': Y_test}
    data_train, data_val, data_test = [X_train, T_train], [X_val, T_val], [X_test, T_test]
    data_generators = create_generators_from_data(
        x_train=data_train, y_train=Y_train, 
        x_val=data_val, y_val=Y_val,
        x_test=data_test, y_test=Y_test
    )
    return data_generators, data

# 2. Define CMNN model

In [3]:
def build_cmnn_ensemble(input_dim, load_weights=False):
    alpha11, alpha12, alpha13 = 1.0, 1.0, 1.0
    alpha21, alpha22, alpha23 = 1.0, 1.0, 1.0
    num_estimators = 2

    ensemble, ensemble_weights = [], {}
    for i in range(num_estimators):
        f11 = ICK(
            kernel_assignment=['ImplicitDenseNetKernel'],
            kernel_params={
                'ImplicitDenseNetKernel':{
                    'input_dim': input_dim,
                    'latent_feature_dim': 512,
                    'num_blocks': 0, 
                    'num_layers_per_block': 1, 
                    'num_units': 512, 
                    'activation': 'relu',
                }
            }
        )
        f12 = ICK(
            kernel_assignment=['ImplicitDenseNetKernel'],
            kernel_params={
                'ImplicitDenseNetKernel':{
                    'input_dim': input_dim,
                    'latent_feature_dim': 512,
                    'num_blocks': 0, 
                    'num_layers_per_block': 1, 
                    'num_units': 512, 
                    'activation': 'relu', 
                }
            }
        )
        f13 = ICK(
            kernel_assignment=['ImplicitDenseNetKernel'],
            kernel_params={
                'ImplicitDenseNetKernel':{
                    'input_dim': input_dim,
                    'latent_feature_dim': 512,
                    'num_blocks': 0, 
                    'num_layers_per_block': 1, 
                    'num_units': 512, 
                    'activation': 'relu', 
                }
            }
        )
        f21 = ICK(
            kernel_assignment=['ImplicitDenseNetKernel'],
            kernel_params={
                'ImplicitDenseNetKernel':{
                    'input_dim': input_dim,
                    'latent_feature_dim': 512,
                    'num_blocks': 0, 
                    'num_layers_per_block': 1, 
                    'num_units': 512, 
                    'activation': 'relu', 
                }
            }
        )
        f22 = ICK(
            kernel_assignment=['ImplicitDenseNetKernel'],
            kernel_params={
                'ImplicitDenseNetKernel':{
                    'input_dim': input_dim,
                    'latent_feature_dim': 512,
                    'num_blocks': 0, 
                    'num_layers_per_block': 1, 
                    'num_units': 512, 
                    'activation': 'relu', 
                }
            }
        )
        f23 = ICK(
            kernel_assignment=['ImplicitDenseNetKernel'],
            kernel_params={
                'ImplicitDenseNetKernel':{
                    'input_dim': input_dim,
                    'latent_feature_dim': 512,
                    'num_blocks': 0, 
                    'num_layers_per_block': 1, 
                    'num_units': 512, 
                    'activation': 'relu', 
                }
            }
        )
        if load_weights:
            for f in ['f11', 'f12', 'f13', 'f21', 'f22', 'f23']:
                eval(f).kernels[0].load_state_dict(torch.load('./checkpoints/cmick_jobs.pt')['model_'+str(i+1)][f])
        else:
            model_weights = {
                'f11': f11.kernels[0].state_dict(), 'f12': f12.kernels[0].state_dict(), 'f13': f13.kernels[0].state_dict(), 
                'f21': f21.kernels[0].state_dict(), 'f22': f22.kernels[0].state_dict(), 'f23': f23.kernels[0].state_dict()
            }
            ensemble_weights['model_'+str(i+1)] = model_weights
        # Set output_binary=True for binary Y0 and Y1
        baselearner = CMICK(
            control_components=[f11,f21], treatment_components=[f12,f22], shared_components=[f13,f23],
            control_coeffs=[alpha11,alpha21], treatment_coeffs=[alpha12,alpha22], shared_coeffs=[alpha13,alpha23], 
            coeff_trainable=True, output_binary=True
        )
        ensemble.append(baselearner)
    if not load_weights:
        if not os.path.exists('./checkpoints'):
            os.makedirs('./checkpoints')
        torch.save(ensemble_weights, './checkpoints/cmick_jobs.pt')

    return ensemble

# 3. Training and evaluation of CMNN model

In [4]:
def fit_and_evaluate_cmnn(ensemble, data_generators, data, lr, treatment_index=1): 
    # The index of "T_train" in "data_train" is 1
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = 'sgd'
    optim_params = {
        'lr': lr, 
        'momentum': 0.99,
        'weight_decay': 1e-4
    }
    epochs, patience = 1000, 10
    trainer = CMICKEnsembleTrainer(
        model=ensemble,
        data_generators=data_generators,
        optim=optim,
        optim_params=optim_params, 
        model_save_dir=None,
        device=device,
        epochs=epochs,
        patience=patience, 
        treatment_index=treatment_index
    )
    trainer.train()
    
    mean_test_pred, std_test_pred, y_test_true = trainer.predict()
    y_test, t_test = data['Y_test'], data['T_test']
    r_pol = policy_risk_binary(mean_test_pred, y_test, t_test)
    print('Policy risk (CMNN):             %.4f' % (r_pol))
    
    return r_pol

# 4. Benchmark 1: original CMGP

In [5]:
def fit_and_evaluate_original_cmgp(data):
    X_train, T_train, Y_train = data['X_train'], data['T_train'], data['Y_train']
    X_test, T_test, Y_test = data['X_test'], data['T_test'], data['Y_test']
    cmgp_model = CMGP(X_train, T_train, Y_train)

    mu0_test_pred, mu1_test_pred = cmgp_model.predict(X_test, return_var=False)
    mu_test_pred = np.concatenate([mu0_test_pred, mu1_test_pred], axis=1)
    r_pol = policy_risk_binary(mu_test_pred, Y_test, T_test)
    print('Policy risk (CMGP):             %.4f' % (r_pol))
    
    return r_pol

# 5. Benchmark 2: CEVAE

In [6]:
def fit_and_evaluate_cevae(data):
    lr = 1e-4
    weight_decay = 1e-4
    batch_size = int(data['X_train'].shape[0]/8)
    train_iters = 20000
    eval_iters = 200
    latent_dim = 20
    n_h = 64
    X_train, T_train, Y_train, X_test = torch.tensor(data['X_train']).float(), torch.tensor(data['T_train']).float(), \
                                        torch.tensor(data['Y_train']).float(), torch.tensor(data['X_test']).float()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    X_train = X_train.to(device)
    T_train = T_train.to(device)
    Y_train = Y_train.to(device)
    X_test = X_test.to(device)
    
    # init networks (overwritten per replication)
    p_x_z_dist = p_x_z(dim_in=latent_dim, nh=3, dim_h=n_h).to(device)
    p_t_z_dist = p_t_z(dim_in=latent_dim, nh=1, dim_h=n_h, dim_out=1).to(device)
    p_y_zt_dist = p_y_zt(dim_in=latent_dim, nh=3, dim_h=n_h, dim_out=1, output_binary=True).to(device)
    q_t_x_dist = q_t_x(dim_in=X_train.shape[1], nh=1, dim_h=n_h, dim_out=1).to(device)

    # t is not feed into network, therefore not increasing input size (y is fed).
    q_y_xt_dist = q_y_xt(dim_in=X_train.shape[1], nh=3, dim_h=n_h, dim_out=1, output_binary=True).to(device)
    q_z_tyx_dist = q_z_tyx(dim_in=X_train.shape[1]+1, nh=3, dim_h=n_h, dim_out=latent_dim).to(device)
    p_z_dist = normal.Normal(torch.zeros(latent_dim).to(device), torch.ones(latent_dim).to(device))

    # Create optimizer
    params = list(p_x_z_dist.parameters()) + \
             list(p_t_z_dist.parameters()) + \
             list(p_y_zt_dist.parameters()) + \
             list(q_t_x_dist.parameters()) + \
             list(q_y_xt_dist.parameters()) + \
             list(q_z_tyx_dist.parameters())

    # Adam is used, like original implementation, in paper Adamax is suggested
    optimizer = optim.Adamax(params, lr=lr, weight_decay=weight_decay)

    # init q_z inference
    q_z_tyx_dist = init_qz(q_z_tyx_dist, Y_train, T_train, X_train).to(device)

    # Training
    loss = []
    for _ in tqdm(range(train_iters)):
        i = np.random.choice(X_train.shape[0],size=batch_size,replace=False)
        Y_train_shuffled = Y_train[i,:].to(device)
        X_train_shuffled = X_train[i,:].to(device)
        T_train_shuffled = T_train[i,:].to(device)

        # inferred distribution over z
        xy = torch.cat((X_train_shuffled, Y_train_shuffled), 1).to(device)
        z_infer = q_z_tyx_dist(xy=xy, t=T_train_shuffled)
        # use a single sample to approximate expectation in lowerbound
        z_infer_sample = z_infer.sample()        

        # RECONSTRUCTION LOSS
        # p(x|z)
        x_con = p_x_z_dist(z_infer_sample)
        # l1 = x_bin.log_prob(x_train).sum(1)

        l2 = x_con.log_prob(X_train_shuffled).sum(1)

        # p(t|z)
        t = p_t_z_dist(z_infer_sample)
        l3 = t.log_prob(T_train_shuffled).squeeze()

        # p(y|t,z)
        # for training use trt_train, in out-of-sample prediction this becomes t_infer
        y = p_y_zt_dist(z_infer_sample, T_train_shuffled)
        l4 = y.log_prob(Y_train_shuffled).squeeze()

        # REGULARIZATION LOSS
        # p(z) - q(z|x,t,y)
        # approximate KL
        l5 = (p_z_dist.log_prob(z_infer_sample) - z_infer.log_prob(z_infer_sample)).sum(1)

        # AUXILIARY LOSS
        # q(t|x)
        t_infer = q_t_x_dist(X_train_shuffled)
        l6 = t_infer.log_prob(T_train_shuffled).squeeze()

        # q(y|x,t)
        y_infer = q_y_xt_dist(X_train_shuffled, T_train_shuffled)
        l7 = y_infer.log_prob(Y_train_shuffled).squeeze()

        # Total objective
        # inner sum to calculate loss per item, torch.mean over batch
        loss_mean = torch.mean(l2 + l3 + l4 + l5 + l6 + l7)
        loss.append(loss_mean.cpu().detach().numpy())
        objective = -loss_mean

        optimizer.zero_grad()
        # Calculate gradients
        objective.backward()
        # Update step
        optimizer.step()
        
    # Evaluation
    Y0_pred, Y1_pred = [], []
    t_infer = q_t_x_dist(X_test)

    eval_iters = 1000
    for _ in tqdm(range(eval_iters)):
        ttmp = t_infer.sample()
        y_infer = q_y_xt_dist(X_test, ttmp)

        xy = torch.cat((X_test, y_infer.sample()), 1)
        z_infer = q_z_tyx_dist(xy=xy, t=ttmp).sample()
        y0 = p_y_zt_dist(z_infer, torch.zeros(z_infer.shape[0],1).to(device)).sample()
        y1 = p_y_zt_dist(z_infer, torch.ones(z_infer.shape[0],1).to(device)).sample()
        Y0_pred.append(y0.detach().cpu().numpy().ravel())
        Y1_pred.append(y1.detach().cpu().numpy().ravel())

    # sameple from the treated and control    
    mu0_test_pred = np.mean(np.array(Y0_pred), axis=0)
    mu1_test_pred = np.mean(np.array(Y1_pred), axis=0)
    mu_test_pred = np.stack([mu0_test_pred, mu1_test_pred], axis=1)
    Y_test, T_test = data['Y_test'], data['T_test']
    
    r_pol = policy_risk_binary(mu_test_pred, Y_test, T_test)
    print('Policy risk (CEVAE):             %.4f' % (r_pol))
    
    return r_pol

# 6. Benchmark 3: GANITE

In [7]:
def fit_and_evaluate_ganite(data):
    X_train, T_train, Y_train = data['X_train'], data['T_train'], data['Y_train']
    X_test, T_test, Y_test = data['X_test'], data['T_test'], data['Y_test']
    model = Ganite(X_train, T_train, Y_train, dim_hidden=4, alpha=1, beta=5, minibatch_size=128, depth=3)
    with torch.no_grad():
        X_test = model._check_tensor(X_test).float()
        y_test_pred = model.inference_nets(X_test).detach().numpy()
    r_pol = policy_risk_binary(y_test_pred, Y_test, T_test)
    print('Policy risk (GANITE):             %.4f' % (r_pol))
    return r_pol

# 7. Benchmark 4: CFRnet

# Main function

In [ ]:
def main():
    train_ratio, test_ratio = 0.56, 0.20
    lr = 1e-4
    data_generators, data = load_and_preprocess_data(train_ratio, test_ratio, random_state=1)
    input_dim = data['X_train'].shape[1]
    ensemble = build_cmnn_ensemble(input_dim, load_weights=False)
    r_pol_cmnn = fit_and_evaluate_cmnn(ensemble, data_generators, data, lr)
    r_pol_cmgp = fit_and_evaluate_original_cmgp(data)
    r_pol_cevae = fit_and_evaluate_cevae(data)
    r_pol_ganite = fit_and_evaluate_ganite(data)

if __name__ == "__main__":
    main()